# Final Model

## Structure

Our final model will include

- XGBoost best classifier submission
- Random Forest classifier
- Logistic regression override *specifically* for predicting the minority class

Let's import some stuff and load the data first.

In [97]:
import xgboost as xgb
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score
from sklearn.metrics import recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import GridSearchCV, validation_curve, learning_curve
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier

In [237]:
def map_installer(installer):
    
    unknown = ['0', 'unknown']
    
    if installer in unknown:
        return 'unknown'
    
    government = ['government ', 'government', 'dwe', 'hesawa', 'rwe', 'central government', 'lga',
                 'district council', 'gover', 'gove', 'gov', 'district water department',
                 'sengerema water department', 'distri', 'centr', 'distric water department',
                 'tasaf']
    
    if installer in government:
        return 'government'
    
    community = ['community', 'commu', 'villagers', 'twesa']
    
    if installer in community:
        return 'community'
    
    religious = ['church of disciples', 'kkkt', 'world vision', 'rc church', 'rc', 'tcrs',
                'dmdd']
    
    if installer in religious:
        return 'religious'
    
    international = ['norad', 'fini water', 'danida', 'danid', 'ces', 'kuwait',
                    'finw']
    
    if installer in international:
        return 'international'
    
    private = ['private', 'privat', 'kiliwater', 'wedeco']
    
    if installer in private:
        return 'private'
    
    aid = ['roman', 'amref', 'world bank', 'unicef', 'oxfam']
    
    if installer in aid:
        return 'aid'
    
    
    return 'other'

def season(month):
    """
    Returns a string corresponding to the typical seasononal period in Tanzania, given
    month input as an integer
    """
    if month in [4,5]:
        return 'heavy_rain'
    elif month in [12, 1, 2]:
        return 'hot_dry'
    elif month == 3:
        return 'intermittent_rain'
    elif month in [6,7,8,9,10]:
        return 'cool_dry'
    else:
        return 'short_rains'

def wrangle(X, X_train):
    """
    Takes in the raw water pump features and returns an enhanced dataframe
    
    We also need to pass in the training data for some functions, such as getting top value_counts.
    
    When wrangling the training set, we will pass two copies of the training data
    
    """
    X = X.copy()
    X_train = X_train.copy()
    
    ### date recorded ###
    X['year_recorded'] = X['date_recorded'].apply(lambda x: int(x.split('-')[0]))
    X['years_since_construction'] = [record - construction if construction > 1900 
                                        else 100 for record, construction in 
                                        zip(X['year_recorded'], X['construction_year'])]
    
    days = X_train['date_recorded'].value_counts()
    X['day_record_count'] = [days[day] if day in days.index else 0 for day in X['date_recorded']]
    X['month_recorded'] = X['date_recorded'].apply(lambda x: int(x.split('-')[1]))
    X['season'] = X['month_recorded'].apply(season)
    
    ### funder / installer ###
    # convert all strings to lowercase
    X['funder'], X['installer'] = X['funder'].str.lower(), X['installer'].str.lower()
    X['funded_and_installed'] = np.where(X['funder'] == X['installer'], True, False)
    # encode into categories
    X['funder_cat'] = X['funder'].apply(map_installer)
    X['installer_cat'] = X['installer'].apply(map_installer)
    
    # drop full feature sets
    X = X.drop(['funder', 'installer'], axis=1)
    
#     top_funders, top_installers = X_train['funder'].str.lower().value_counts().index[:20], X_train['installer'].str.lower().value_counts().index[:20]
#     X['funder'] = ['other' if funder not in top_funders else funder for funder in X['funder']]
#     X['installer'] = ['other' if installer not in top_installers else installer for installer in X['installer']]
    
    ### total static head ###
    X['tsh_zero'] = X['amount_tsh'] == 0.0
#     X['log_amount_tsh'] = np.log(X['amount_tsh']) <- has problems with 0
    
    ### waterpoint name ###
    X['wpt_name'] = X['wpt_name'].str.lower()
    top_wpt = X_train['wpt_name'].str.lower().value_counts().index[:50]
    X['wpt_name'] = ['other' if wpt_name not in top_wpt else wpt_name for wpt_name in X['wpt_name'].str.lower()]
    
    ### population ###
    X['population_unknown'] = X['population'] == 0
    X['population'].replace(0, np.nan, inplace=True)
    X["population"].fillna(X_train.groupby(['region', 'district_code'])["population"].transform("median"), inplace=True)

    X["population"].fillna(X_train.groupby(['region'])["population"].transform("median"), inplace=True)

    X["population"].fillna(X_train["population"].median(), inplace=True)

    
    ### subvillage ###
    # encode unknown as binary
    X['subvillage_unknown'] = X['subvillage'].isnull()
    # encode top 50, everything else as other
    X['subvillage'] = X['subvillage'].str.lower()
    sub_villages_count = X_train['subvillage'].str.lower().value_counts()
    X['subvillage_waterpoints'] = [sub_villages_count[vill] if vill in sub_villages_count.index else 0 for vill in X['subvillage'].str.lower()]
    
    top_subvillage = X_train['subvillage'].str.lower().value_counts().index[:50]
    X['subvillage'] = ['other' if vill not in top_subvillage else vill for vill in X['subvillage'].str.lower()]
    
    ### region code ###
    # encode region codes as strings, which will be automatically one hot encoded later
    X['region_code'] = X['region_code'].astype(str)
    X = X.drop('region', axis=1)
    
    ### district code ###
    # encode district codes as strings, which will be automatically one hot encoded later
    X['district_code'] = X['district_code'].astype(str)
    
    ### lga ###
    X['lga'] = X['lga'].str.lower()
    # encode urban and rural
    X['lga_rural'] = [lga.find('rural') != -1 for lga in X['lga']]
    X['lga_urban'] = [lga.find('urban') != -1 for lga in X['lga']]
    # encode anything except the top 30 as other
    top_lga = X_train['lga'].str.lower().value_counts().index[:30]
    X['lga'] = ['other' if lga not in top_lga else lga for lga in X['lga'].str.lower()]
    
    ### ward ###
    # create new variable with the number of waterpoints in a given ward
    ward_counts = X_train['ward'].str.lower().value_counts()
    X['ward_wpt_count'] = [ward_counts[ward] if ward in ward_counts.index else 0 for ward in X['ward'].str.lower()]
    
    
    ### public meeting ###
    X['public_meeting_unknown'] = X['public_meeting'].isnull() # encode missing vals to a binary var
    X['public_meeting'] = X['public_meeting'].fillna(False) # fill na with false
    
    ### recorded by ###
    X = X.drop('recorded_by', axis=1) # drop it, all the same value
    
    ### permit ###
    X['permit_unknown'] = X['permit'].isnull() # encode missing vals to a binary var
    X['permit'] = X['permit'].fillna(False) # fill na with false
    
    ### scheme name/management ###
    X['scheme_management'] = X['scheme_management'].fillna('None') # encode nulls as 'None'
    X = X.drop('scheme_name', axis=1)
    
    ### construction year ###
    X['construction_unknown'] = X['construction_year'] < 1900
#     X['early60s'] = X['construction_year'].between(1900, 1964)
#     X['late60s'] = X['construction_year'].between(1965, 1969)
#     X['early70s'] = X['construction_year'].between(1970, 1974)
#     X['late70s'] = X['construction_year'].between(1975, 1979)
#     X['early80s'] = X['construction_year'].between(1980, 1984)
#     X['late80s'] = X['construction_year'].between(1985, 1989)
#     X['early90s'] = X['construction_year'].between(1990, 1994)
#     X['late90s'] = X['construction_year'].between(1995, 1999)
#     X['early00s'] = X['construction_year'].between(2000, 2004)
#     X['late00s'] = X['construction_year'].between(2005, 2009)
#     X['early10s'] = X['construction_year'].between(2010, 2014)
#     X = X.drop('construction_year', axis=1)
    
    ### extraction type ###
    X = X.drop(['extraction_type_group', 'extraction_type_class'], axis=1)
    
    ### management ###
    X = X.drop('management_group', axis=1)
    
    ### payment ###
    X = X.drop('payment_type', axis=1)
    
    ### quality ###
    X = X.drop(['quality_group', 'quantity_group'], axis=1)
    
    ### source ###
    X = X.drop(['source_type', 'source_class'], axis=1)
    
    ### waterpoint type ###
    X = X.drop('waterpoint_type_group', axis=1)
    
    ### longitude/latitude/gps_height ###
    X['gps_height_bad'] = X['gps_height'] <= 0.0 # min height should be sea level
    X['latitude_bad'] = X['latitude'] < 25.0
    X['longitude_bad'] = X['longitude'] > -0.5
    
    # fill in missing values based on location - STOLEN FROM BREADWARD
    
#     training_data["gps_height"].fillna(training_data.groupby(['region', 'district_code'])["gps_height"].transform("mean"), inplace=True)
#     training_data["gps_height"].fillna(training_data.groupby(['region'])["gps_height"].transform("mean"), inplace=True)
#     training_data["gps_height"].fillna(training_data["gps_height"].mean(), inplace=True)
#     training_data["amount_tsh"].fillna(training_data.groupby(['region', 'district_code'])["amount_tsh"].transform("median"), inplace=True)
#     training_data["amount_tsh"].fillna(training_data.groupby(['region'])["amount_tsh"].transform("median"), inplace=True)
#     training_data["amount_tsh"].fillna(training_data["amount_tsh"].median(), inplace=True)
#     training_data["latitude"].fillna(training_data.groupby(['region', 'district_code'])["latitude"].transform("mean"), inplace=True)
#     training_data["longitude"].fillna(training_data.groupby(['region', 'district_code'])["longitude"].transform("mean"), inplace=True)
#     training_data["longitude"].fillna(training_data.groupby(['region'])["longitude"].transform("mean"), inplace=True)
    
    
    
    ### DROPPING AFTER FEATURE ENGINEERING ####
    X = X.drop('date_recorded', axis=1)
  
    return X # returned wrangled dataframe





## XGBoost

Next, let's load in our best XGBoost model and run predictions.

In [238]:
# data prep for XGBoost
df_raw = pd.read_csv('train_features.csv', index_col=0)
df_test_raw = pd.read_csv('test_features.csv', index_col=0)
X_train, X_test = wrangle(df_raw, df_raw), wrangle(df_test_raw, df_raw)
y_train = pd.read_csv('train_labels.csv', index_col=0)
# encoding categoricals
cat_encoder = ce.OneHotEncoder(use_cat_names=True)
cat_encoder.fit(X_train, y_train)
X_train_clean = cat_encoder.transform(X_train)
X_test_clean = cat_encoder.transform(X_test)
# prepping all data, we should only be using dtest for predictions
# encode labels
from sklearn.preprocessing import LabelEncoder
xgb_le = LabelEncoder()
y_train_clean = xgb_le.fit_transform(y_train.values.ravel())

X_train_clean_train, X_train_clean_val, y_train_clean_train, y_train_clean_val = train_test_split(X_train_clean,
                                                                                                 y_train_clean,
                                                                                                 test_size=0.10,
                                                                                                 stratify=y_train_clean,
                                                                                                 shuffle=True,
                                                                                                 random_state=420)

dtrain = xgb.DMatrix(X_train_clean_train, label=y_train_clean_train)
dval = xgb.DMatrix(X_train_clean_val, label=y_train_clean_val)
dtest = xgb.DMatrix(X_test_clean)


In [302]:
# load model 
# v19 best iteration thusfar
xgb_best = xgb.Booster(model_file='xgboost_iteration21_v0.model')
# predict validation and test data
xgb_test, xgb_val = xgb_best.predict(dtest), xgb_best.predict(dval)

## Random Forest Classifier

Let's implement a random forest classifier to compliment our other models.

In [314]:
# load and wrangle data
df_raw = pd.read_csv('train_features.csv', index_col=0)
df_test_raw = pd.read_csv('test_features.csv', index_col=0)
X_train, X_test = wrangle(df_raw, df_raw), wrangle(df_test_raw, df_raw)
y_train = pd.read_csv('train_labels.csv', index_col=0)

#  splitting data - ONE SUBMISSION ON FULL TRAINING SET
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                 y_train,
                                                 test_size=0.10,
                                                 stratify=y_train,
                                                 shuffle=True,
                                                 random_state=420)

In [347]:
# best params

# # best params - no overfitting
# Random Forest Training Accuracy 0.8507482229704452
# Random Forest Validation Accuracy 0.8053872053872054
# rf = RandomForestClassifier(n_estimators=2000, 
#                            criterion='entropy',
#                            max_depth=50, 
#                            min_samples_leaf = 2 
#                            )

# initialize and fit
rf = RandomForestClassifier(n_estimators=3000, 
                           criterion='entropy',
                           max_depth=50, 
                           min_samples_leaf = 3 # 2 keeps the tree from overfitting, 1 is spicy
                           )
pipe_rf = make_pipeline(ce.OneHotEncoder(use_cat_names=True),
                       rf).fit(X_train, y_train)

/Users/zach/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


In [348]:
rf_train = pipe_rf.predict(X_train)
rf_val = pipe_rf.predict(X_val)
rf_test = pipe_rf.predict(X_test)

In [349]:
print ('Random Forest Training Accuracy', accuracy_score(y_train, rf_train))
print ('Random Forest Validation Accuracy', accuracy_score(y_val, rf_val))

Random Forest Training Accuracy 0.8200149644594089
Random Forest Validation Accuracy 0.7936026936026936


In [350]:
rf_val_prob = pipe_rf.predict_proba(X_val)
rf_test_prob = pipe_rf.predict_proba(X_test)

## Validation and Kaggle Submission

### Validation

Lastly, we want to evaluate our model on validation data, and submit to Kaggle if appropriate.

In [328]:
# predict with XGB
xgb_val_classes = xgb_le.inverse_transform(np.asarray([np.argmax(line) for line in xgb_val]))
xgb_test_classes = xgb_le.inverse_transform(np.asarray([np.argmax(line) for line in xgb_test]))

In [329]:
# compare classification reports for xgb alone vs. combined
y_val = xgb_le.inverse_transform(y_train_clean_val.ravel())
print ('XGBoost Classification Report:\n', classification_report(y_val, xgb_val_classes))
print ('Accuracy Scores\n')
print ('XGBoost Accuracy:', accuracy_score(y_val, xgb_val_classes))

XGBoost Classification Report:
                          precision    recall  f1-score   support

             functional       0.82      0.90      0.86      3226
functional needs repair       0.67      0.35      0.46       432
         non functional       0.84      0.80      0.82      2282

              micro avg       0.82      0.82      0.82      5940
              macro avg       0.78      0.68      0.71      5940
           weighted avg       0.82      0.82      0.81      5940

Accuracy Scores

XGBoost Accuracy: 0.8217171717171717


In [344]:
# XGboost (xgb_val) + rf predictions (rf_val_prob) - test on validation
ensemble_val = np.sum([xgb_val, rf_val_prob], axis=0)
ensemble_val_pred = xgb_le.inverse_transform(np.asarray([np.argmax(line) for line in ensemble_val]))

In [345]:
print ('XGBoost + RF Accuracy:', accuracy_score(y_val, ensemble_val_pred))
print ('XGBoost + RF Classification Report\n', classification_report(y_val, ensemble_val_pred))

XGBoost + RF Accuracy: 0.8198653198653199
XGBoost + RF Classification Report
                          precision    recall  f1-score   support

             functional       0.80      0.92      0.86      3226
functional needs repair       0.77      0.28      0.41       432
         non functional       0.85      0.78      0.82      2282

              micro avg       0.82      0.82      0.82      5940
              macro avg       0.81      0.66      0.69      5940
           weighted avg       0.82      0.82      0.81      5940



In [332]:
# predict test data
ensemble_test = np.sum([xgb_test, rf_test_prob], axis=0)
ensemble_test_pred = xgb_le.inverse_transform(np.asarray([np.argmax(line) for line in ensemble_test]))

### Kaggle Submission

In [309]:
df_submit = pd.DataFrame(data=ensemble_test_pred, index=X_test.index, columns=['status_group'])
df_submit.to_csv('xgboost_rf_v6.csv')

In [310]:
# SUBMIT!
%env KAGGLE_CONFIG_DIR=/Users/zach/Kaggle
!kaggle competitions submit -c ds1-predictive-modeling-challenge -f xgboost_rf_v6.csv -m 'xgboost classifier and random forest classifier, rf trained on validation data too'



env: KAGGLE_CONFIG_DIR=/Users/zach/Kaggle
100%|█████████████████████████████████████████| 264k/264k [00:01<00:00, 232kB/s]
Successfully submitted to DS1 Predictive Modeling Challenge